**Import needed packages**

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import load_model
import random
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
import seaborn as sns

 **Load and prepare data**

In [ ]:
datas = np.load('validation_bw1.npy')
datan = datas

data_s = datas.reshape(datas.shape[0], 128, 336, 1)
data_n = datan.reshape(datan.shape[0], 128, 336, 1)

labelsn = np.load('numbers_labels_val.npy')
labelss = np.load('suits_labels_val.npy')

one_hot_lab_s = np.eye(5)[labelss - 14] #convert suit labels to one hot encoding

unique_lab_n = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18]) #vector of unique labels
num_classes = len(unique_lab_n)
label_to_index = {label: index for index, label in enumerate(unique_lab_n)} #dic to associate indeces to labels
label_indices_n = np.array([label_to_index[label] for label in labelsn]) # convert original labels to indices

one_hot_lab_n = np.eye(num_classes)[label_indices_n] #convert num labels to one hot encoding


**Do infrence on loaded data**

In [ ]:
model_s = load_model('suits_model_original.h5')

pred_lab_s = model_s.predict(datas)
pred_classes = np.argmax(pred_lab_s, axis=1)
true_classes = np.argmax(one_hot_lab_s, axis=1)

misclassified_indices = np.where(pred_classes != true_classes)[0]
misclassified_images = datas[misclassified_indices]
misclassified_true_labels = true_classes[misclassified_indices]
misclassified_pred_labels = pred_classes[misclassified_indices]

wrong_ass = len(misclassified_indices)
print("you have",wrong_ass,"misclassified samples")

for i in range(wrong_ass):

    plt.imshow(misclassified_images[i], cmap='gray') #plot misclass. suits samples
    plt.title(f'True label: {misclassified_true_labels[i]+1}, Predicted label: {misclassified_pred_labels[i]+1}')
    plt.show()

cms = confusion_matrix(true_classes, pred_classes)
cms_normalized = cms.astype('float') / cms.sum(axis=1)[:, np.newaxis]

suit_classes = ['Spades', 'Clubs','Hearts','Diamonds','Joker']

plt.figure(figsize=(8, 6))
sns.heatmap(cms_normalized, annot=True, cmap='Blues', fmt='.2f', xticklabels=suit_classes, yticklabels=suit_classes) #plot suits confusion matrix
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for suits class.')
plt.show()

In [ ]:
model_n = load_model("numbers_model_original.h5")

pred_lab_n = model_n.predict(datan)
pred_classes_ = np.argmax(pred_lab_n, axis=1)
true_classes_ = np.argmax(one_hot_lab_n, axis=1)

misclassified_indices_ = np.where(pred_classes_ != true_classes_)[0]
misclassified_images_ = datan[misclassified_indices_]
misclassified_true_labels_ = true_classes_[misclassified_indices_]
misclassified_pred_labels_ = pred_classes_[misclassified_indices_]

for i in range(len(misclassified_indices_)):
    plt.imshow(misclassified_images_[i], cmap='gray')#plot misclass. numbers samples
    plt.title(f'True label: {misclassified_true_labels_[i]+1}, Predicted label: {misclassified_pred_labels_[i]+1}')
    plt.show()

cmn = confusion_matrix(true_classes_, pred_classes_)
cmn_n = cmn.astype('float') / cmn.sum(axis=1)[:, np.newaxis]

num_classes = ['A','2','3','4','5','6','7','8','9','10','J','Q','K','Joker']

plt.figure(figsize=(8, 6))
sns.heatmap(cmn_n, annot=True, cmap='Blues', fmt='.2f', xticklabels=num_classes, yticklabels=num_classes) #plot numbers confusion matrix
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix for numbers class.')
plt.show()